In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [269]:
data=pd.read_csv(r'C:\Users\Praca\Desktop\Statystyka praktyczna\Kod PL\dane\house_sales.csv')

In [270]:
data.drop(columns=['Unnamed: 0','DocumentDate','PropertyID', 'AdjSalePrice','ImpsVal'], inplace=True) # deleting unneccesary data

In [273]:
data.rename(columns={'SalePrice': 'Target', 'ym': 'Date_of_cons'}, inplace=True)


In [262]:
X_columns=[i for i in data.columns if i != 'Target']
Y_value=data['Target']
X_values=data[X_columns]
X_values.insert(0,'one_col',1)

In [263]:
X_values['Date_of_cons']=pd.to_datetime(X_values['Date_of_cons'])
X_values=pd.get_dummies(X_values, drop_first=True) # creating dummy varibles 
X_values.NewConstruction=[1 if i==True else 0 for i in X_values.NewConstruction] # boolen to int 

In [264]:
data_day=pd.to_datetime('today')
days=pd.to_datetime(data_day.date())-X_values.Date_of_cons
X_values['Days_from_cons']=days.dt.days
X_values.drop(columns='Date_of_cons', inplace=True) #changing date type to int; Linear regression treats date as random number

In [265]:
X_values.dtypes

one_col                         int64
zhvi_px                         int64
zhvi_idx                      float64
NbrLivingUnits                  int64
SqFtLot                         int64
SqFtTotLiving                   int64
SqFtFinBasement                 int64
Bathrooms                     float64
Bedrooms                        int64
BldgGrade                       int64
YrBuilt                         int64
YrRenovated                     int64
TrafficNoise                    int64
LandVal                         int64
ZipCode                         int64
NewConstruction                 int64
PropertyType_Single Family      uint8
PropertyType_Townhouse          uint8
Days_from_cons                  int64
dtype: object

In [274]:
X_values.corr() # correlation matrix

,one_col,zhvi_px,zhvi_idx,NbrLivingUnits,SqFtLot,SqFtTotLiving,SqFtFinBasement,Bathrooms,Bedrooms,BldgGrade,YrBuilt,YrRenovated,TrafficNoise,LandVal,ZipCode,NewConstruction,PropertyType_Single Family,PropertyType_Townhouse,Days_from_cons
one_col,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zhvi_px,NaN,1.000000,1.000000,0.015245,-0.012901,-0.021590,-0.007247,-0.027789,-0.000330,-0.045961,-0.016092,-0.000766,0.008455,-0.023783,0.004230,0.038360,-0.002020,-0.005775,0.660711
zhvi_idx,NaN,1.000000,1.000000,0.015245,-0.012901,-0.021590,-0.007247,-0.027789,-0.000330,-0.045961,-0.016092,-0.000766,0.008455,-0.023783,0.004230,0.038360,-0.002020,-0.005775,0.660711
NbrLivingUnits,NaN,0.015245,0.015245,1.000000,-0.005113,0.059563,0.066803,0.111227,0.168076,-0.048095,-0.109173,0.011371,0.083209,0.031421,0.005786,-0.036871,-0.274532,-0.030500,0.015431
SqFtLot,NaN,-0.012901,-0.012901,-0.005113,1.000000,0.195950,0.035421,0.107436,0.069076,0.145501,0.068406,0.006912,-0.012827,0.071051,-0.010942,-0.067399,0.099317,-0.097779,-0.026242
SqFtTotLiving,NaN,-0.021590,-0.021590,0.059563,0.195950,1.000000,0.407127,0.764186,0.600288,0.770493,0.311597,0.067160,-0.060718,0.467805,-0.031595,0.113968,0.176949,-0.194454,-0.006194
SqFtFinBasement,NaN,-0.007247,-0.007247,0.066803,0.035421,0.407127,1.000000,0.272684,0.312405,0.137570,-0.210897,0.109488,0.053026,0.283728,0.016431,-0.113575,0.082863,-0.104529,-0.017331
Bathrooms,NaN,-0.027789,-0.027789,0.111227,0.107436,0.764186,0.272684,1.000000,0.537953,0.658772,0.468643,0.066460,-0.060015,0.309340,-0.028770,0.197179,-0.066429,0.047955,-0.002594
Bedrooms,NaN,-0.000330,-0.000330,0.168076,0.069076,0.600288,0.312405,0.537953,1.000000,0.367995,0.148205,0.045941,-0.039173,0.204193,-0.019309,0.011165,0.175537,-0.235194,0.003853
BldgGrade,NaN,-0.045961,-0.045961,-0.048095,0.145501,0.770493,0.137570,0.658772,0.367995,1.000000,0.434642,0.025352,-0.063295,0.460108,-0.029064,0.168762,0.000333,0.023973,-0.024567


In [275]:
high_corr={} # finding and droping columns with correlation above 0.8
for index in X_values.corr().index:
    for column in X_values.corr().columns:
        if X_values.corr().loc[index, column] > 0.8:
            if index!= column:
                if X_values.corr().loc[index, column] not in high_corr.keys():
                
                    high_corr[X_values.corr().loc[index, column]]=[index,column]
high_corr
X_values.drop(columns=[values[1] for values in high_corr.values()], inplace=True)

In [276]:
model1=LinearRegression()

In [277]:
model1.fit(X_values, Y_value)

LinearRegression()

In [278]:
model1_pred=pd.DataFrame(Y_value).astype(float)
model1_pred['Y_pred']=model1.predict(X_values).round(2)
model1_pred['Epsilon^2']=(model1_pred['Target']-model1_pred['Y_pred'])**2
model1_pred['(y-y_mean)^2']=(model1_pred['Target']-model1_pred['Target'].mean())**2
Fi_sq=model1_pred['Epsilon^2'].sum()/model1_pred['(y-y_mean)^2'].sum()*100
Fi_sq
R_sq=100-Fi_sq

In [279]:
R_sq

81.19140132687907

R_sq shows that 81,19% of the total volatility of the empirical value of the housing price (Target) was explained in the model by the formation of exogenous variables (X_values)

Conclusion: we have good quality of data.


In [247]:
from scipy import stats

In [280]:
T= len(Y_value) # amount of rows
K=len(X_columns) # amount of X varibles
t_st=stats.t.interval(0.95, T-K) # t-Students

In [249]:
std_sq1=1/(T-K)*model1_pred['Epsilon^2'].sum().round(2)
std_sq1
std1=np.sqrt(std_sq1).round(2)
std1 # standard deviation

150391.03

Examination of the significance of regression parameters.


In [250]:
dic={ name: coef.round(2) for name, coef in zip( X_values, model1.coef_)  }
dic

{'one_col': 0.0,
 'zhvi_px': 1.12,
 'NbrLivingUnits': -18768.0,
 'SqFtLot': 0.23,
 'SqFtTotLiving': 120.64,
 'SqFtFinBasement': -22.65,
 'Bathrooms': 28655.05,
 'Bedrooms': -24092.83,
 'BldgGrade': 50762.02,
 'YrBuilt': -960.49,
 'YrRenovated': -7.05,
 'TrafficNoise': 3724.26,
 'LandVal': 1.1,
 'ZipCode': 1.78,
 'NewConstruction': 34502.08,
 'PropertyType_Single Family': -364.79,
 'PropertyType_Townhouse': 63104.35,
 'Days_from_cons': 4.66}

Now i have to find Average error of estimation.

1.Finding matrix (X'X)^-1, where X is matrix with X_values data.

2.Finding Average error of estimation.

In [251]:
matrix=X_values.copy()

matrix=matrix.to_numpy()
matrix_transpone=matrix.transpose()
matrix_multi=np.dot(matrix_transpone,matrix)
inv_matrix=np.linalg.inv(matrix_multi) # LSM 

In [252]:
AEE=[std1*np.sqrt(inv_matrix[i][i])  for i in range(len(inv_matrix))] #average error of estimation

In [282]:
ES_data=pd.DataFrame(X_values.columns,columns=['X_columns'])
ES_data['coef']= model1.coef_
ES_data['AEE']=AEE
ES_data['t_emp']=ES_data['coef']/ES_data['AEE']
ES_data['is_signif']=[ True if t_emp > t_st[1] or t_emp < t_st[0] else False for t_emp in ES_data['t_emp']] 
# checking if  given parametr is significant
ES_data

,X_columns,coef,AEE,t_emp,is_signif
0,one_col,0.000000,195164.959816,0.000000,False
1,zhvi_px,1.121958,0.036643,30.618624,True
2,NbrLivingUnits,-18768.000763,10764.680812,-1.743480,False
3,SqFtLot,0.226633,0.035529,6.378886,True
4,SqFtTotLiving,120.637115,2.618856,46.064816,True
5,SqFtFinBasement,-22.649248,2.842901,-7.966950,True
6,Bathrooms,28655.049881,2334.630187,12.273914,True
7,Bedrooms,-24092.832482,1475.060619,-16.333452,True
8,BldgGrade,50762.019025,1561.806547,32.502117,True
9,YrBuilt,-960.491987,49.602801,-19.363664,True


Examination of the significance of regression parameters shows that all False parameters in 'is_signif' column do not have significant inpact on final output.

In [283]:
X_values2=X_values[ES_data[ES_data.is_signif==True].X_columns.to_numpy()] # creating dataset without insignificant parameters

In [255]:
model2=LinearRegression()

In [256]:
model2.fit(X_values2,Y_value)

LinearRegression()

In [284]:
model2_pred=pd.DataFrame(Y_value).astype(float)
model2_pred['Y_pred']=model2.predict(X_values2).round(2)
model2_pred['Epsilon^2']=(model2_pred['Target']-model2_pred['Y_pred'])**2
model2_pred['(y-y_mean)^2']=(model2_pred['Target']-model2_pred['Target'].mean())**2
Fi_sq2=model2_pred['Epsilon^2'].sum()/model2_pred['(y-y_mean)^2'].sum()*100
Fi_sq2
R_sq2=100-Fi_sq2

In [285]:
R_sq2-R_sq # removing of some columns did not change quality of data

-0.007591024447506811

In [294]:
std_sq2=1/(T-K)*model2_pred['Epsilon^2'].sum().round(2)
std_sq2
std2=np.sqrt(std_sq2).round(2)
std2-std1 # difference between std of first and second models is around 30$

30.35000000000582

In [293]:
RMSE=np.sqrt(model2_pred['Epsilon^2'].sum()/T)
RMSE*100/Y_value.mean()

29.603064224161045

Summary:

1.Above model explains 81,19% of the total volatility of the empirical value of the housing price (Target) was explained in the model by the formation of exogenous variables (X_values)- good quality of data.

2.RMSE= 150 000 USD this indicator shows that the actual price of flats differs from the forecast by an average of 150 000 USD which is virtually 30% of its price (low accuracy) .

